In [1]:
# GRAIN_TYPE = 'Wheat'
# GRAIN_TYPE = 'Oats'
# GRAIN_TYPE = 'Barley'
# GRAIN_TYPE = 'Sorghum'
GRAIN_TYPE = 'Soybeans'
# GRAIN_TYPE = 'Corn'

In [2]:
#Import libraries
#Import libraries
import requests
import pydot
import pandas as pd


#Data visualization
import seaborn as sn
import matplotlib.pyplot as plt

#Data Manipulation
import pandas as pd
import numpy as np

# Machine Learning
from sklearn.datasets import make_regression
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


# Helper functions

In [3]:
def plot_line(metric, title, xlabel):
    plt.figure(figsize=(8,3))
    plt.title(title, fontsize = 16)
    plt.plot(metric)
    plt.xlabel(xlabel, fontsize = 14)
    plt.grid()
    plt.legend(loc= "best")
    plt.show()

def scatter_plot(trueValues, predictions, title):
  plt.figure(figsize=(8,3))
  ax = plt.axes()
  maxVal = max( max(trueValues), max(predictions) )

  ax.scatter(predictions, trueValues)
  ax.plot([0, 1, maxVal], [0, 1, maxVal], label="Ideal fit")
  print('Maxval here is: ', maxVal)
  plt.title(title, fontsize = 16)
  plt.xlabel("Predictions", fontsize = 14)
  plt.ylabel("Real", fontsize = 14)
  plt.grid()

def calculate_r_squared(y_true, y_pred):
   corr_matrix = np.corrcoef(y_true, y_pred)
   corr = corr_matrix[0,1]
   R_sq = corr**2
   return R_sq

In [4]:
#url dataset
URL = "../../Datasets/processed/" + GRAIN_TYPE + ".csv"

#read in excel format
df = pd.read_csv(URL)

print("From USDA: ", URL)

df.head()

From USDA:  ../../Datasets/processed/Soybeans.csv


,Unnamed: 0,Variety,Freq,d(cm),M%,Density,Attn,Phase,Phase_Corr,Permittivity_real,Permittivity_imaginary
0,0,GA COOK,9.0,7.7,10.17,0.7016,9.0939,-117.925,-477.925,2.474,0.227
1,1,GA COOK,10.0,7.7,10.17,0.7016,10.4287,-172.315,-532.315,2.479,0.234
2,2,GA COOK,11.0,7.7,10.17,0.7016,11.3727,137.970,-582.030,2.468,0.232
3,3,GA COOK,12.0,7.7,10.17,0.7016,12.1813,83.069,-636.931,2.474,0.228
4,4,GA COOK,13.0,7.7,10.17,0.7016,13.0729,30.688,-689.312,2.472,0.226


In [5]:
# Data summary
df.describe()

,Unnamed: 0,Freq,d(cm),M%,Density,Attn,Phase,Phase_Corr,Permittivity_real,Permittivity_imaginary
count,571.000000,571.000000,571.000000,571.000000,571.000000,571.000000,571.000000,571.000000,571.000000,571.000000
mean,285.000000,10.851138,7.438529,13.801646,0.763736,17.357792,-4.944697,-660.636492,2.895324,0.465377
std,164.977776,3.567973,1.158926,3.697234,0.044452,5.806705,102.237880,206.374962,0.374272,0.244705
min,0.000000,5.000000,4.400000,7.570000,0.681300,8.002300,-179.356000,-1294.972000,2.323000,0.131000
25%,142.500000,8.000000,7.700000,10.510000,0.720150,12.468500,-90.374500,-809.184500,2.611500,0.267000
50%,285.000000,11.000000,7.700000,13.420000,0.770400,16.712900,-11.039000,-648.691000,2.794000,0.404000
75%,427.500000,13.000000,7.700000,15.970000,0.801300,21.723700,83.099000,-495.564500,3.122500,0.586000
max,570.000000,18.000000,10.500000,22.170000,0.855400,29.897000,179.751000,-257.753000,4.339000,1.399000


In [6]:
# Convert to numpy array
df_features = df[['Freq', 
                    'd(cm)', 
                    'Attn', 
                    'Phase', 
                    'Phase_Corr', 
                    'Permittivity_real', 
                    'Permittivity_imaginary']]

# df_targets = df[['M%', 'Density']]
df_targets = df[['Density']]

dataset_x = df_features.to_numpy()
dataset_y = df_targets.to_numpy()

In [7]:
# Perform train-test split on the normalized features
X_train, X_test, y_train, y_test = train_test_split(dataset_x, dataset_y, 
                                                    test_size=0.2, 
                                                    random_state=42)

# Define Hyperparameter Grid

In [8]:
param_grid = {
    'pca__n_components': [4,5],  # Number of principal components to keep
    'svr__kernel': ['linear', 'rbf'],  # Kernel type for SVR
    'svr__C': [0.1, 1.0, 10.0],  # Regularization parameter for SVR
    'svr__gamma': ['scale', 'auto'],  # Kernel coefficient for 'rbf' kernel
}

# Apply PCA

In [9]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import make_pipeline

# Create the PCA-SVR Model
pca = PCA()
svr = SVR()
pca_svr = RandomizedSearchCV(make_pipeline(pca, svr), 
                             param_distributions=param_grid, 
                             n_iter=10,
                             cv=5,
                             verbose=1)

# Perform Grid Search and Fit the Model
pca_svr.fit(X_train, y_train)

# Get the Best Hyperparameters
best_params = pca_svr.best_params_
print("Best Hyperparameters:", best_params)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Hyperparameters: {'svr__kernel': 'linear', 'svr__gamma': 'auto', 'svr__C': 0.1, 'pca__n_components': 5}


/home/zeenat/anaconda3/envs/venv_ml/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zeenat/anaconda3/envs/venv_ml/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zeenat/anaconda3/envs/venv_ml/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zeenat/anaconda3/envs/venv_ml/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was 

In [10]:
# Make predictions on the test set
y_pred = pca_svr.predict(X_test)

y_pred_bd = y_pred

In [15]:
# Calculate Mean Squared Error for each target
mse_bd = mean_squared_error(y_test, y_pred_bd)
print("Mean Squared Error for Target 1:", mse_bd)

r_2_bd = calculate_r_squared(y_true=y_test,
                             y_pred=y_pred_bd)
print("R Squared Error for Target 1:", r_2_bd)

# rmse_bd = rmse = np.sqrt(mean_squared_error(y_true=y_test,
#                              y_pred=y_pred_bd))
# print("RMSE for Target 1:", rmse_bd)

# mse_mc = mean_squared_error(y_test[:, 1], y_pred_mc)
# print("Mean Squared Error for Target 2:", mse_mc)

Mean Squared Error for Target 1: 0.002102328413043479


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1 and the array at index 1 has size 115

In [ ]:
# BULK DENSITY
scatter_plot(trueValues=y_test[:, 1], 
             predictions=y_pred_bd, 
             title="Bulk Density")
plt.xlim([min(min(y_pred_bd), min(y_test[:, 1]))-0.1, max(max(y_pred_bd), max(y_test[:, 1]))+0.1])

a, b = np.polyfit(x = y_pred_bd, y = y_test[:, 1], deg=1) # y = ax + b
x_best_fit = np.arange(0, max(max(y_pred_bd), max(y_test[:, 1]))+0.1, 0.1)
plt.plot(x_best_fit, a*x_best_fit + b, c='red', label='Best fit')
plt.legend()